/* PART 2: SQLite
/* We now want you to jump over to a local instance of the database on your machine. 

Copy and paste the LocalSQLConnection.py script into an empty Jupyter notebook, and run it. 

Make sure that the SQLFiles folder containing thes files is in your working directory, and
that you haven't changed the name of the .db file from 'sqlite\db\pythonsqlite'.

You should see the output from the initial query 'SELECT * FROM FACILITIES'.

Complete the remaining tasks in the Jupyter interface. If you struggle, feel free to go back
to the PHPMyAdmin interface as and when you need to. 

You'll need to paste your query into value of the 'query1' variable and run the code block again to get an output.

In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect("sqlite_db_pythonsqlite.db")
# cur = conn.cursor()
# cur.execute("SELECT * FROM Facilities")
# results = cur.fetchall()

In [3]:
# Q10: Produce a list of facilities with a total revenue less than 1000.
#      The output of facility name and total revenue, sorted by revenue. 
#      Remember that there's a different cost for guests and members!

query = '''SELECT f.name AS facility,
            SUM(CASE WHEN Bookings.memid <> 0 THEN f.membercost * Bookings.slots
    	            ELSE f.guestcost * Bookings.slots END) AS revenue
            FROM Facilities AS f
            INNER JOIN Bookings ON f.facid = Bookings.facid
            GROUP BY 1 HAVING revenue < 1000
            ORDER BY revenue DESC;'''


df = pd.read_sql_query(query, conn)
df.style.hide_index()

facility,revenue
Pool Table,270
Snooker Table,240
Table Tennis,180


In [4]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order 

query = '''SELECT m.firstname || ' ' || m.surname AS 'new member',
		  m1.firstname || ' ' || m1.surname AS 'recommended by'
        FROM Members as m
        INNER JOIN Members as m1 
        ON m.recommendedby = m1.memid
        WHERE m1.memid <> 0
        ORDER BY m.surname, m.firstname;'''

df = pd.read_sql_query(query, conn)
df.style.hide_index()

new member,recommended by
Florence Bader,Ponder Stibbons
Anne Baker,Ponder Stibbons
Timothy Baker,Jemima Farrell
Tim Boothe,Tim Rownam
Gerald Butters,Darren Smith
Joan Coplin,Timothy Baker
Erica Crumpet,Tracy Smith
Nancy Dare,Janice Joplette
Matthew Genting,Gerald Butters
John Hunt,Millicent Purview


In [5]:
# Q12: Find the facilities with their usage by member, but not guests 

query = '''
SELECT f.name, b.total AS usage_by_member
FROM (SELECT facid, name 
      FROM Facilities) AS f
LEFT JOIN (SELECT facid, COUNT(*) AS total
           FROM Bookings as b
           WHERE memid <> 0
           GROUP BY facid) AS b 
           ON f.facid = b.facid
ORDER BY usage_by_member DESC;'''


df = pd.read_sql_query(query, conn)
df.style.hide_index()

name,usage_by_member
Pool Table,783
Massage Room 1,421
Snooker Table,421
Table Tennis,385
Badminton Court,344
Tennis Court 1,308
Tennis Court 2,276
Squash Court,195
Massage Room 2,27


In [6]:
# Q13: Find the facilities usage by month, but not guests

query = '''
SELECT strftime('%m', b.starttime) as Month, f.name, SUM(b.slots) AS monthly_usage
        FROM Bookings AS b
        LEFT JOIN Facilities AS f 
        ON f.facid = b.facid
        WHERE b.memid <> 0
        GROUP BY f.name, month
        ORDER BY month;
      '''


df = pd.read_sql_query(query, conn)
df.to_string(index=False)
df.style.hide_index()

Month,name,monthly_usage
07,Badminton Court,165
07,Massage Room 1,166
07,Massage Room 2,8
07,Pool Table,110
07,Snooker Table,140
07,Squash Court,50
07,Table Tennis,98
07,Tennis Court 1,201
07,Tennis Court 2,123
08,Badminton Court,414
